In [ ]:
import cma
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import time
import imageio
import os
from matplotlib import cm
%matplotlib inline

## Helper functions

In [ ]:
def scale(x, bounds):
    """Scale the input numbers in [0, 1] to the range of each variable"""
    if bounds.ndim == 1:
        return bounds[0] + x * (bounds[1] - bounds[0])
    else:
        return bounds[:, 0] + x * (bounds[:, 1] - bounds[:, 0])

def normalize(x_scaled, bounds):
    if bounds.ndim == 1:
        return (x_scaled - bounds[0]) / (bounds[1] - bounds[0])
    else:
        return (x_scaled - bounds[:, 0]) / (bounds[:, 1] - bounds[:, 0])

In [ ]:
import signal_tl as stl

def compute_stl_rob(phi, signal_builder, record):
    time_index = np.arange(len(record))
    signal = signal_builder(record, time_index)
    rob = stl.compute_robustness(phi, signal)
    return rob.at(0)

## Robustness Analyzer

In [ ]:
class RobustnessAnalysis:
    def __init__(self, env_builder, agent, episode_eval, delta_0, dev_bounds, dist, options=None):
        self.env_builder = env_builder
        self.agent = agent
        self.episode_eval = episode_eval
        self.delta_0 = delta_0
        self.dev_bounds = dev_bounds
        self.dist = dist
        self.options = {    
            'epsilon': 1e-2,
            'deviation_restarts': 2,
            'deviation_sigma': 0.2, # normalized
            'deviation_timeout': np.inf, # timeout in minutes for each CMA run for finding a deviation
            'deviation_evals': 100, # max number of evaluations

            'falsification_sigma': 0.4, # a bigger sigma for falsify STL
            'falsification_timeout': np.inf,
            'falsification_restarts': 1,
            'falsification_episodes': 100,

            'episodes_of_each_x0': 1,
            'steps_of_each_x0': 200,
        }
        if options is not None:
            self.options.update(options)
            
        self.cache = {}
    
    def robustness_boundary(self):
        delta, _ = self.min_unsafe_deviation()
        boundary = self.dist(delta, self.delta_0) - self.options['epsilon']
        while True:
            delta, _ = self.min_unsafe_deviation(boundary)
            if delta is None:
                break
            boundary = self.dist(delta, self.delta_0) - self.options['epsilon']
        return boundary
    
    def any_unsafe_deviation(self, boundary=None):
        delta = None
        delta_dist = np.inf
        
        bipop = False
        num_tries = self.options['deviation_restarts']
        sigma = self.options['deviation_sigma']
        timeout = self.options['deviation_timeout']
        evals = self.options['deviation_evals']
        
        objective = lambda delta: self.deviated_sys_eval(delta)[0]
        if boundary is not None:
            constraints = lambda delta: [self.dist(delta, self.delta_0) - boundary]
            
            def random_x0():
                x0 = normalize(self.delta_0, self.dev_bounds)
                x0_r = np.random.normal(x0, boundary/2)
                return np.clip(x0_r, 0.0, 1.0)
            
            for i in range(1):
                print(f'\n================ Any unsafe deviation trial {i+1} ==============>')
                
                cfun = cma.ConstrainedFitnessAL(
                    lambda x: objective(scale(x, self.dev_bounds)),
                    lambda x: constraints(scale(x, self.dev_bounds)),
                    find_feasible_first=True
                )
                _, es = cma.fmin2(
                    cfun,
                    random_x0, # x0,
                    sigma,
                    {'bounds': [0.0, 1.0], 'tolstagnation': 0, 'tolx': 1e-4, 'timeout': timeout * 60,
                     'ftarget': 0.0, 'maxfevals': evals},
                    callback=cfun.update,
                    restarts=num_tries,
                    bipop=bipop
                )
                print("=============== CMA Results:=================>")
                print(es.result)
                print("=============================================>")
                if cfun.best_feas.info is not None:
                    delta = scale(cfun.best_feas.info['x'], self.dev_bounds)
                    delta_dist = self.dist(delta, self.delta_0)
                    break
        else:
            for i in range(1):
                print(f'\n================ Any unsafe deviation trial {i+1} ==============>')
                
                _, es = cma.fmin2(
                    lambda x: objective(scale(x, self.dev_bounds)),
                    lambda: np.random.rand(len(self.delta_0)), # x0,
                    sigma,
                    {'bounds': [0.0, 1.0], 'tolstagnation': 0, 'tolx': 1e-4, 'timeout': timeout * 60,
                     'ftarget': 0.0, 'maxfevals': evals},
                    restarts=num_tries,
                    bipop=bipop
                )
                print("=============== CMA Results:=================>")
                print(es.result)
                print("=============================================>")
                if es.result.fbest < 0.0:
                    delta = scale(es.result.xbest, self.dev_bounds)
                    delta_dist = self.dist(delta, self.delta_0)
                    break
        
        return delta, delta_dist
            
    
    def min_unsafe_deviation(self, boundary=None):
        min_dist = np.inf
        min_delta = None

        num_tries = self.options['deviation_restarts']
        sigma = self.options['deviation_sigma']
        timeout = self.options['deviation_timeout']
        evals = self.options['deviation_evals']

        objective = lambda delta: self.dist(delta, self.delta_0)

        def random_x0():
            x0 = normalize(self.delta_0, self.dev_bounds)
            x0_r = np.random.normal(x0, sigma/2)
            return np.clip(x0_r, 0.0, 1.0)
        
        Xss = []
        for i in range(num_tries+1):
            print(f'\n================ Min unsafe deviation trial {i+1} ==============>')
            
            if boundary is not None:
                constraints = lambda delta: [self.deviated_sys_eval(delta)[0],
                                             self.dist(delta, self.delta_0) - boundary]
            else:
                constraints = lambda delta: [self.deviated_sys_eval(delta)[0]]

            cfun = cma.ConstrainedFitnessAL(
                lambda x: objective(scale(x, self.dev_bounds)),
                lambda x: constraints(scale(x, self.dev_bounds)),
                find_feasible_first=True
            )
            es = cma.CMAEvolutionStrategy(
                random_x0(),
                sigma,
                {'bounds': [0.0, 1.0], 'tolstagnation': 0, 'tolx': 1e-4, 'timeout': timeout * 60,
                 'maxfevals': evals}
            )
            Xs = []
            while not es.stop():
                X = es.ask()
                Xs.extend(X)
                es.tell(X, [cfun(x) for x in X])
                cfun.update(es)
                
#             _, es = cma.fmin2(
#                 cfun,
#                 random_x0, # x0,
#                 sigma,
#                 {'bounds': [0.0, 1.0], 'tolstagnation': 0, 'tolx': 1e-4, 'timeout': timeout * 60},
#                 callback=cfun.update,
#                 restarts=num_tries,
#                 bipop=bipop
#             )

            Xss.append(np.array(Xs))
            print("=============== CMA Results:=================>")
            print(es.result)
            print("=============================================>")
            
            if cfun.best_feas.info is not None:
                print(cfun.best_feas.info)
                delta = scale(cfun.best_feas.info['x'], self.dev_bounds)
                delta_dist = self.dist(delta, self.delta_0)
                if delta_dist < min_dist:
                    min_dist = delta_dist
                    min_delta = delta

        return min_delta, min_dist, np.array(Xss)
    
    def deviated_sys_eval(self, delta):
        bipop = False
        sigma = self.options['falsification_sigma']
        timeout = self.options['falsification_timeout']
        num_tries = self.options['falsification_restarts']
        max_episodes = self.options['falsification_episodes']

        env, x0_bounds = self.env_builder(delta)
        objective = lambda x: self.prop_eval(env, x)
        
        min_f = np.inf
        min_x = None
        for i in range(1):
            x, es = cma.fmin2(
                lambda x: objective(scale(x, x0_bounds)),
                lambda: np.random.rand(len(x0_bounds)),
                sigma,
                {'bounds': [0.0, 1.0], 'maxfevals': max_episodes, 'timeout': timeout * 60, 'verbose': -9},
                restarts=num_tries,
                bipop=bipop
            )
            if es.result.fbest < min_f:
                min_f = es.result.fbest
                min_x = x
        
        env.close()
        self.cache[tuple(delta)] = (min_f, scale(min_x, x0_bounds))
        return self.cache[tuple(delta)]
    
    def prop_eval(self, env, x0):
        space = env.observation_space
        
        model_reset = self.agent['model_reset']
        next_action = self.agent['next_action']
        
        num_tries = self.options['episodes_of_each_x0']
        max_episode_steps = self.options['steps_of_each_x0']
        
        values = []
        for i in range(num_tries):
            obs = env.reset_to(x0)
            obs_record = [obs]
            reward_record = [0]
            
            if model_reset is not None:
                model_reset()
            for step in range(max_episode_steps):
                action = next_action(obs)
                obs, reward, _, _ = env.step(action)
                obs_record.append(np.clip(obs, space.low, space.high))
                reward_record.append(reward)
            
            v = self.episode_eval(np.asarray(obs_record), np.asarray(reward_record))
            values.append(v)
        
        # Use the min robustness of multiple runs at an initial state x
        return np.asarray(values).min()
    
    def visualize_deviation(self, delta, gif):
        value, x0 = self.cache[tuple(delta)]
        env, _ = self.env_builder(delta)
        self.visual_episode(env, x0, save_gif=gif)
        env.close()
        print("STL robustness value:", value)
        print("Initial state:", x0)
    
    def init_fig(self, env):
        plt.figure()
        plt.title(env.spec.id)
        plt.axis('off')
        return plt.imshow(env.render(mode='rgb_array'))
    
    def update_fig(self, img, env, step, reward, done, episode_measure_name, value):
        title = plt.title(
            f"{env.spec.id}\n" +
            f"Step: {step} | Reward: {reward:.3f} | Done: {done}\n" +
            f"{episode_measure_name}: {value:.3f}"
        )
        if done or value < 0:
            plt.setp(title, color='r')
        else:
            plt.setp(title, color='k')
        img.set_data(env.render(mode='rgb_array'))
        fig = plt.gcf()
        display.display(fig)
        display.clear_output(wait=True)

        return np.asarray(fig.canvas.buffer_rgba())

    def visual_episode(self, env, x0=None, visualize_in_notebook=True,
                       sleep=0.01, save_gif=None, episode_measure_name='STL'):
        if x0 is not None:
            obs = env.reset_to(x0)
        else:
            obs = env.reset()

        model_reset = self.agent['model_reset']
        next_action = self.agent['next_action']
        max_episode_steps = self.options['steps_of_each_x0']

        if model_reset is not None:
            model_reset()

        if save_gif is not None:
            gif = []
        else:
            gif = None

        # initialize figure in notebook
        if visualize_in_notebook:
            img = self.init_fig(env)

        space = env.observation_space
        total_reward = 0.0
        obs_record = [obs]
        reward_record = [0]

        for step in range(1, max_episode_steps+1):
            action = next_action(obs)
            obs, reward, done, info = env.step(action)
            total_reward += reward
            obs_record.append(np.clip(obs, space.low, space.high))
            reward_record.append(reward)

            if visualize_in_notebook:
                v = self.episode_eval(np.asarray(obs_record), np.asarray(reward_record))
                fig_data = self.update_fig(img, env, step, total_reward, done, episode_measure_name, v)
                if save_gif is not None:
                    gif.append(fig_data)
            else:
                env.render()

            if sleep > 0.0:
                time.sleep(sleep)

        if save_gif is not None:
            imageio.mimsave(save_gif, [data for data in gif], fps=10)
    
    def grid_data(self, x_bound, y_bound, n_x, n_y, x_name="X", y_name="Y", z_name="Z",
                  grid_name="grid_data", override=False, out_dir='data'):
        if not os.path.exists(f'{out_dir}/{grid_name}.csv') or override:
            if not os.path.exists(out_dir):
                os.mkdir(out_dir)
                
            X = np.linspace(x_bound[0], x_bound[1], n_x)
            Y = np.linspace(y_bound[0], y_bound[1], n_y)
            X, Y = np.meshgrid(X, Y, indexing='ij')

            grid_data = np.zeros((n_x, n_y))

            for i in range(n_x):
                for j in range(n_y):
                    # treat xv[i,j], yv[i,j]
                    x, y = X[i, j], Y[i, j]
                    robustness, _ = self.deviated_sys_eval([x, y])
                    grid_data[i, j] = robustness

            np.savetxt(f"{out_dir}/{x_name}.csv", X, delimiter=",")
            np.savetxt(f"{out_dir}/{y_name}.csv", Y, delimiter=",")
            np.savetxt(f"{out_dir}/{grid_name}.csv", grid_data, delimiter=",")
        else:
            X = np.loadtxt(f"{out_dir}/{x_name}.csv", delimiter=",")
            Y = np.loadtxt(f"{out_dir}/{y_name}.csv", delimiter=",")
            grid_data = np.loadtxt(f"{out_dir}/{grid_name}.csv", delimiter=",")
        
        return X, Y, grid_data
        
    
    def grid_plot(self, x_bound, y_bound, n_x, n_y, x_name="X", y_name="Y", z_name="Z",
                  grid_name="grid_data", override=False, out_dir='data', boundary=None):
        X, Y, grid_data = self.grid_data(x_bound, y_bound, n_x, n_y, x_name, y_name, z_name,
                                         grid_name, override, out_dir)

        fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(14, 14))
        
        if boundary is not None:
            mask = np.asarray([
                [self.dist([x, y], self.delta_0) > boundary for x, y in zip(xs, ys)]
                for xs, ys in zip(X, Y)
            ])
            ax.plot_surface(X, Y, np.ma.masked_where(mask, grid_data), cmap=cm.coolwarm)
            ax.plot_surface(X, Y, grid_data, cmap=cm.coolwarm, alpha=0.25)
        else:
            ax.plot_surface(X, Y, grid_data, cmap=cm.coolwarm)
        
        ax.set_zlabel(z_name, fontsize=13)
        ax.set_xlabel(x_name, fontsize=13)
        ax.set_ylabel(y_name, fontsize=13)

        return ax, X, Y, grid_data
    
    def heatmap(self, x_bound, y_bound, n_x, n_y, x_name="X", y_name="Y", z_name="Z",
                grid_name="grid_data", override=False, out_dir='data', boundary=None):
        X, Y, grid_data = self.grid_data(x_bound, y_bound, n_x, n_y, x_name, y_name, z_name,
                                         grid_name, override, out_dir)
        
        fig, ax = plt.subplots()
        im = ax.imshow(grid_data.T, cmap=cm.coolwarm)
        
        ax.set_xticks(np.arange(0, len(X[:, 0]), 3), labels=['{:.2f}'.format(x) for x in X[:, 0][::3]])
        ax.set_yticks(np.arange(0, len(Y[0]), 3), labels=['{:.2f}'.format(y) for y in Y[0][::3]])
        cbar = ax.figure.colorbar(im)
        cbar.ax.set_ylabel(z_name, rotation=-90, va="bottom")

        ax.set_xlabel(x_name)
        ax.set_ylabel(y_name)

        if boundary is not None:
            center = normalize(self.delta_0, self.dev_bounds)
            c_X = np.linspace(center[0] - boundary, center[0] + boundary, 100)
            c_Y1 = center[1] + np.sqrt(boundary**2 - (c_X - center[0])**2)
            c_Y2 = center[1] - np.sqrt(boundary**2 - (c_X - center[0])**2)

            ax.scatter(center[0] * n_x, center[1] * n_y, color='black')
            ax.plot(c_X * n_x, c_Y1 * n_y, color='black')
            ax.plot(c_X * n_x, c_Y2 * n_y, color='black')

        plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
        
        return ax, X, Y, grid_data

In [ ]:
class RADecorator:
    def __init__(self, ra):
        self.ra = ra
    
    def __getattr__(self, name):
        return getattr(self.ra, name)

In [ ]:
class EarlyStop(RADecorator):
    # define the function to override
    def min_unsafe_deviation(self, boundary=None, early_stop_threshold=3):
        print('================ Using early stop optimization! =================>')
        
        min_dist = np.inf
        min_delta = None

        num_tries = self.options['deviation_restarts']
        sigma = self.options['deviation_sigma']
        timeout = self.options['deviation_timeout']
        evals = self.options['deviation_evals']

        objective = lambda delta: self.dist(delta, self.delta_0)

        def random_x0():
            x0 = normalize(self.delta_0, self.dev_bounds)
            x0_r = np.random.normal(x0, sigma/2)
            return np.clip(x0_r, 0.0, 1.0)
        
        constraint_w = 1
        Xss = []
        for i in range(num_tries+1):
            print(f'\n================ Min unsafe deviation trial {i+1} ==============>')
            print('Constraints weight:', constraint_w, 'Boundary:', boundary)
            
            # Optimization: if CMA tends to move away from the best feasible, increase the weight of the constraints
            if boundary is not None:
                constraints = lambda delta: [self.deviated_sys_eval(delta)[0] * constraint_w,
                                             (self.dist(delta, self.delta_0) - boundary) * constraint_w]
            else:
                constraints = lambda delta: [self.deviated_sys_eval(delta)[0] * constraint_w]

            cfun = cma.ConstrainedFitnessAL(
                lambda x: objective(scale(x, self.dev_bounds)),
                lambda x: constraints(scale(x, self.dev_bounds)),
                find_feasible_first=True
            )
            es = cma.CMAEvolutionStrategy(
                random_x0(),
                sigma,
                {'bounds': [0.0, 1.0], 'tolstagnation': 0, 'tolx': 1e-4, 'timeout': timeout * 60,
                 'maxfevals': evals}
            )
            Xs = []
            early_stop = 0
            while not es.stop():
                X = es.ask()
                Xs.extend(X)
                es.tell(X, [cfun(x) for x in X])
                cfun.update(es)
                
                if not cfun.finding_feasible: # Optimization: early stop when the current mean and std is far from the current best feasible
                    cur_best = cfun.best_feas.info['x']
                    cur_mean = np.mean(X, axis=0)
                    if es.sigma < self.dist(scale(cur_best, self.dev_bounds), scale(cur_mean, self.dev_bounds)):
                        early_stop += 1
                    else: # consecutive
                        early_stop = 0
                    if early_stop > early_stop_threshold:
                        print("Current mean and std is far from the existing best solution, early stop!")
                        constraint_w = 10 if constraint_w == 1 else constraint_w + 10 # Optimization: when early stop increase the constraints weight
                        break

            Xss.append(np.array(Xs))
            print("=============== CMA Results: =================>")
            print(es.result)
            print("==============================================>")
            
            if cfun.best_feas.info is not None:
                print(cfun.best_feas.info)
                delta = scale(cfun.best_feas.info['x'], self.dev_bounds)
                delta_dist = self.dist(delta, self.delta_0)
                if delta_dist < min_dist:
                    min_dist = delta_dist
                    min_delta = delta
                    
                    # Optimization: for every restart, add the current min distance as a constraint
                    boundary = min_dist

        return min_delta, min_dist, np.array(Xss)

In [ ]:
class MeanOfCMASamples(RADecorator):
    def __init__(self, ra):
        self.ra = ra
        self.show_msg = True
    
    def deviated_sys_eval(self, delta):
        if self.show_msg:
            print('==================== Using mean of CMA samples! ======================>')
            self.show_msg = False
        
        sigma = self.options['falsification_sigma']
        timeout = self.options['falsification_timeout']
        num_tries = self.options['falsification_restarts']
        max_episodes = self.options['falsification_episodes']

        env, x0_bounds = self.env_builder(delta)
        objective = lambda x: self.prop_eval(env, x)
        
        min_f = np.inf
        min_x = None
        Ys = []
        for _ in range(num_tries+1):
            es = cma.CMAEvolutionStrategy(
                np.random.rand(len(x0_bounds)),
                sigma,
                {'bounds': [0.0, 1.0], 'maxfevals': max_episodes, 'timeout': timeout * 60, 'verbose': -9}
            )
            while not es.stop():
                X = es.ask()
                Y = [objective(scale(x, x0_bounds)) for x in X]
                Ys.extend(Y)
                es.tell(X, Y)

            if es.result.fbest < min_f:
                min_f = es.result.fbest
                min_x = es.result.xbest
        
        env.close()
        self.cache[tuple(delta)] = (np.mean(Ys), scale(min_x, x0_bounds))
        
        return self.cache[tuple(delta)]

In [ ]:
from datetime import datetime

class MeanOfRandomSamples(RADecorator):
    def __init__(self, ra):
        self.ra = ra
        self.show_msg = True
        
    def deviated_sys_eval(self, delta):
        if self.show_msg:
            print('===================== Using mean of random samples ========================>')
            self.show_msg = False
        
        num_tries = self.options['falsification_restarts']
        max_episodes = self.options['falsification_episodes'] * (num_tries + 1)
        timeout = self.options['falsification_timeout'] * (num_tries + 1)
        
        start = datetime.now()
        env, x0_bounds = self.env_builder(delta)
        evals = []
        for _ in range(max_episodes):
            x0 = np.random.uniform(x0_bounds[:, 0], x0_bounds[:, 1])
            evals.append(self.prop_eval(env, x0))
            if (datetime.now() - start).total_seconds() > timeout * 60:
                break
        
        env.close()
        return np.mean(evals), None